In [4]:
import pandas as pd
data = pd.read_csv('Twitter_Data.csv')

In [5]:
y = data['category']
X = data['clean_text']

In [6]:
print(y.shape)
print(X.shape)
print(type(X[1]))

(162980,)
(162980,)
<class 'str'>


In [7]:
def mapNanValues(data):
    nan_values = []
    for i in range(len(data)):
        if type(data[i]) == float:
            nan_values.append(i)
    return nan_values

In [8]:
import numpy as np
import tensorflow as tf

In [9]:
values = mapNanValues(X)
print(values)
y = y.drop(index=values).reset_index(drop=True)
X = X.drop(index=values).reset_index(drop=True)
print(y.shape)
print(X.shape)
zeros = np.zeros((len(y), 3))
##### One Hot labels ####
for i in range(len(y)):
    if y[i] == -1:
        zeros[i][0] = 1
    elif y[i] == 0:
        zeros[i][1] = 1
    elif y[i] == 1:
        zeros[i][2] = 1

[148, 158694, 159443, 160560]
(162976,)
(162976,)


In [10]:
from tensorflow.keras.layers import LSTM, Dense, Softmax, Dropout, Input, Embedding, TextVectorization
from tensorflow.keras.metrics import CategoricalAccuracy, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras import Model
#from tensorflow.keras.preprocessing.text import one_hot
#from tensorflow.keras.preprocessing.sequence import pad_sequences
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [11]:
def Bert_LSTM():
    text_input = Input(shape=(), dtype=tf.string, name='text')
    preprocessed_text = bert_preprocess(text_input)
    outputs = bert_encoder(preprocessed_text)
    l = Dropout(0.1)(outputs['pooled_output'])
    l = Dense(128, activation='relu')(l)
    l = Dropout(0.1)(l)
    l = Dense(32)(l)
    output = Dense(3, activation='softmax')(l)
    model = Model(inputs=[text_input], outputs = [output])
    return model

In [12]:
def Trainable_Embedding_LSTM(data):
    intencoding = TextVectorization(max_tokens=999, output_mode='int', output_sequence_length=40)
    intencoding.adapt(data)
    intencoding.compile()
    text_input = Input(shape=(), dtype=tf.string, name='text')
    textvec = intencoding(text_input)
    l = Embedding(input_dim=999, output_dim=100)(textvec)
    l = LSTM(128, input_shape=(40, 100), activation='relu', return_sequences=True)(l)
    l = Dropout(0.1)(l)
    l = LSTM(128, activation='relu', return_sequences=False)(l)
    l = Dropout(0.1)(l)
    l = Dense(32)(l)
    output = Dense(3, activation='softmax')(l)
    model = Model(inputs=[text_input], outputs = [output])
    return model


In [13]:
y = zeros

In [15]:
 from sklearn import model_selection
 X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.3, random_state=35)

In [16]:
METRICS = [
      CategoricalAccuracy(name='accuracy'),
      Precision(name='precision'),
      Recall(name='recall')
]

model2 = Bert_LSTM()
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=METRICS, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
history_bert = model2.fit(X_train, y_train, batch_size = 1024, epochs=50, verbose=1)
model2.save('bert_model.model')

In [ ]:
from matplotlib import pyplot as plt
plt.plot(history_bert.history['accuracy'])
plt.plot(history_bert.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Bert_Plot.png', format='png')

NameError: name 'history_bert' is not defined

In [ ]:
model3 = Trainable_Embedding_LSTM(X)
model3.compile(optimizer='adam', loss='categorical_crossentropy', validation_split=0.2, metrics=METRICS, callbacks=[EarlyStopping(monitor='val_loss', patience=5)])
history_custom = model3.fit(X_train, y_train, batch_size = 1024, epochs=50, verbose=1)
model3.save('custom_model.model')

In [ ]:
plt.plot(history_custom.history['accuracy'])
plt.plot(history_custom.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
plt.savefig('Bert_Plot.png', format='png')